In [ ]:
import spacy
import numpy as np
import pandas as pd 
from spacy.lang.en.stop_words import STOP_WORDS
import string

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')


In [ ]:
train=train[['text','target']]

In [ ]:
train

In [ ]:
train['target'].value_counts()

In [ ]:
##Word Counts

In [ ]:
train['word_counts']=train['text'].apply(lambda x:len(str(x).split()))

In [ ]:
train

In [ ]:
###Character counts

In [ ]:
train['char_count']=train['text'].apply(lambda x:len(x))

In [ ]:
train

In [ ]:
###Average Words

In [ ]:
def get_avg_word_len(x):
    words=x.split()
    word_len=0
    for word in words:
        word_len=word_len+len(word)
    return word_len/len(words)

In [ ]:
train['avg_word_count']=train['text'].apply(lambda x: get_avg_word_len(x))

In [ ]:
train

In [ ]:
import re
import unicodedata

In [ ]:
##Remove E-mail

In [ ]:
train['text']=train['text'].apply(lambda x:re.sub(r'([a-zA-Z0-9+_-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)','',x))

In [ ]:
train

In [ ]:
train['text']=train['text'].apply(lambda x:str(x).lower())

In [ ]:
train

In [ ]:
##Remove URL

In [ ]:
train['text']=train['text'].apply(lambda x:re.sub(r"http\S+", "", x))

In [ ]:
train

In [ ]:
##Remove Punctuation

In [ ]:
train['text']=train['text'].apply(lambda x:re.sub('[^A-Z a-z 0-9-]+','',x))

In [ ]:
train

In [ ]:
##Remove multiple Spaces

In [ ]:
train['text']=train['text'].apply(lambda x:" ".join(x.split()))

In [ ]:
train

In [ ]:
###Remove Accented Char

In [ ]:
def remove_acc_char(x):
    x=unicodedata.normalize('NFKD',x).encode('ascii','ignore').decode('utf-8','ignore')
    return x

In [ ]:
train['text']=train['text'].apply(remove_acc_char)

In [ ]:
train

In [ ]:
###Spacy and NLP

In [ ]:
##Remove Stopwords

In [ ]:
train['text']=train['text'].apply(lambda x:" ".join([t for t in x.split() if t not in STOP_WORDS]))

In [ ]:
train

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
##Coverting word into base form(lemmatization)

In [ ]:
x="A letter has been written, asking him to be released"

In [ ]:
def make_base(x):
    x_list=[]
    doc=nlp(x)

    for token in doc:
      lemma=str(token.lemma_)
      if lemma=='-PRON-':
        lemma=token.text
      x_list.append(lemma)
    print(" ".join(x_list))

In [ ]:
make_base(x)

In [ ]:
##Common word removal

In [ ]:
text1=' '.join(train['text'])

In [ ]:
text1=text1.split()

In [ ]:
freq_comm=pd.Series(text1).value_counts()

In [ ]:
f20=freq_comm[:20]

In [ ]:
f20

In [ ]:
#train['text']=train['text'].apply(lambda x:" ".join([t for t in x.split() if t not in f20]))

In [ ]:
train

In [ ]:
##Rare word Removal

In [ ]:
rare=freq_comm[freq_comm.values==1]

In [ ]:
rare

In [ ]:
#Apply same as common word removal

In [ ]:
##Word Cloud Visualization

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
wc=WordCloud(width=1200,height=800).generate(' '.join(train['text']))
plt.imshow(wc)
plt.axis('off')
plt.show()

In [ ]:
##Spelling Correction

In [ ]:
from textblob import TextBlob

In [ ]:
##Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer(ngram_range=(1,1))
text_count=cv.fit_transform(train['text'])

In [ ]:
text_count.toarray()

In [ ]:
##TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(max_df=0.5,min_df=2,ngram_range=(1,2))
x_tfidf=tfidf.fit_transform(train['text'])

In [ ]:
x_tfidf.toarray().shape

In [ ]:
##Word Embedding

In [ ]:
#Spacy Word2vec

In [ ]:
#!python -m spacy download en_core_web_lg

In [ ]:
#nlp=spacy.load('en_core_web_lg')

In [ ]:
from sklearn import model_selection
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,f1_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
lgr=LogisticRegression(random_state=42,max_iter=200)
svm=LinearSVC(random_state=42,max_iter=200)
rfc=RandomForestClassifier(random_state=42,n_jobs=-1,n_estimators=200)
xgb=XGBClassifier(random_state=42,max_depth=7,n_estimators=200)

In [ ]:
clf={'SVM':svm,'RFC':rfc,'XGB':xgb,'LGR':lgr}

In [ ]:
y=train['target']


In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
X=scaler.fit_transform(pd.DataFrame(x_tfidf.toarray()))

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.20,stratify=y)
for key in clf.keys():
          clf[key].fit(X_train,y_train)
          y_pred=clf[key].predict(X_test)
          f1=f1_score(y_test,y_pred)
          print(key,"----",f1)
          

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers import Dense,Dropout,LSTM,SpatialDropout1D

In [ ]:
X=[d.split() for d in train['text'].tolist()]

In [ ]:
tokeniser=Tokenizer()
tokeniser.fit_on_texts(X)

In [ ]:
X=tokeniser.texts_to_sequences(X)

In [ ]:
plt.hist([len(x) for x in X], bins=700)
plt.show()

In [ ]:
nos=np.array([len(x) for x in X])
len(nos[nos>16])

In [ ]:
sent_length=16
embed_docs=pad_sequences(X,padding='pre',maxlen=sent_length)

In [ ]:
vocab_size=len(tokeniser.word_index)+1

In [ ]:
embedding_vector_features=32
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
                                
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(120,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(40))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
X_final=np.array(embed_docs)
Y_final=np.array(y)

In [ ]:
x_train,x_test,Y_Train,Y_Test=train_test_split(X_final,Y_final,test_size=0.2,random_state=42)

In [ ]:
model.fit(x_train,Y_Train,validation_data=(x_test,Y_Test),epochs=10,batch_size=64)

In [ ]:
Y_pred=model.predict_classes(x_test)

In [ ]:
accuracy_score(Y_Test,Y_pred)

In [ ]:
print(classification_report(Y_Test,Y_pred))

In [ ]:
def predict(tweet):
    test_word = tweet
    tw = tokeniser.texts_to_sequences([test_word])
    tw = pad_sequences(tw,padding='pre',maxlen=16)
    prediction = int(model.predict(tw).round().item())
    if(prediction==0):
        return('No Disaster')
    else:
        return('Disaster')

In [ ]:
predict('Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all')

In [ ]:
predict('Damage to school bus on 80 in multi car crash #BREAKING')

In [ ]:
predict('I love fruits')